In [80]:
# MRT_station_name: str
# station_id: int
# station_lat: decimal
# station_lng: decimal
# city_code: str


# Bike_station_name: str
# district: str
# city_code: str
# total_space: int
# station_lat: decimal
# station_lng: decimal
# aval_bike: int
# aval_space: int
# station_status: str
# date: time
# hour: int
# weekend: str
# distance: int


In [38]:
import pandas as pd
import os
import geopy.distance
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [39]:
mrt_station = pd.read_csv('../MRT/mrt_station.csv')



In [40]:
# MRT_station_name: str
# city_code: str
# district: str
# station_lat: decimal
# station_lng: decimal

mrt_station = mrt_station[['StationName.Zh_tw','LocationCityCode', 'LocationTown','StationPosition.PositionLon',
       'StationPosition.PositionLat']]
mrt_station = mrt_station.rename(
    {
        'StationName.Zh_tw':'MRT_station_name',
        'LocationCityCode':'city_code',
        'LocationTown':'district',
        'StationPosition.PositionLat':'station_lat',
        'StationPosition.PositionLon':'station_lng'
    },axis=1)

mrt_station = mrt_station[['MRT_station_name','city_code','district','station_lat','station_lng']]

In [41]:
mrt_station.to_csv('0514_pipe_mrt_station.csv',index=False)

In [42]:
tpe_station = pd.read_csv('bike_usage_realtime.csv',index_col=0)

In [43]:
# Bike_station_name: str
# district: str
# city_code: str
# total: int
# station_lat: decimal
# station_lng: decimal
# distance_to_mrt: int




In [44]:
bike_station = pd.read_csv('../bike/bike_usage_realtime.csv',index_col=0)
bike_station['sarea'] = bike_station['sarea'].replace('臺大公館校區','大安區')
bike_station = bike_station.rename(
    {
        'sna':'Bike_station_name',
        'sarea':'district',
        'latitude':'station_lat',
        'longitude':'station_lng'

    }
    ,axis=1)
bike_station['city_code']='TPE'

bike_station['Bike_station_name'] = bike_station['Bike_station_name'].apply(lambda x: x.split('_')[1])
bike_station = bike_station[['Bike_station_name','district',
                             'city_code','total','station_lat','station_lng']]

In [46]:
mrt_dist = []
for idx, row in bike_station.iterrows():
    point_1 = row[['station_lat','station_lng']].to_numpy()
    mrt_dist.append(int(mrt_station[['station_lat','station_lng']].apply(lambda row:
                                 geopy.distance.geodesic(point_1,row.to_numpy()).m,
                                 axis=1).min()))
bike_station['distance_to_mrt'] = mrt_dist
del mrt_station

In [47]:
bike_station.head()

,Bike_station_name,district,city_code,total,station_lat,station_lng,distance_to_mrt
0,捷運科技大樓站,大安區,TPE,28,25.02605,121.54360,15
1,復興南路二段273號前,大安區,TPE,21,25.02565,121.54357,53
2,國北教大實小東側門,大安區,TPE,16,25.02429,121.54124,302
3,和平公園東側,大安區,TPE,11,25.02351,121.54282,296
4,辛亥復興路口西北側,大安區,TPE,16,25.02153,121.54299,510


In [145]:
ubike_rt = pd.read_csv('ubike_0501.csv',index_col=0)
ubike_rt.drop_duplicates(inplace=True)
ubike_rt['sarea'] = ubike_rt['sarea'].replace('臺大公館校區','大安區')
ubike_rt['sna'] = ubike_rt['sna'].apply(lambda x: x.split('_')[1])


In [146]:
ubike_rt['thres'] = np.min(np.column_stack(((ubike_rt['tot']*0.1).to_numpy(),np.full(len(ubike_rt),5))),axis=1)
def bike_status(row):
    space = row['bemp']
    bike = row['sbi']
    thres = row['thres']
    if space == 0:
        return '車位滿載'
    elif bike ==0:
        return '無車可借'
    elif space <thres:
        return '有車少位'
    elif bike <thres:
        return '少車可借'
    else:
        return '正常租借'

In [147]:
ubike_rt['station_status'] = ubike_rt.apply(bike_status,axis=1)

In [148]:
ubike_rt.columns

Index(['ar', 'aren', 'bemp', 'infoDate', 'infoTime', 'lat', 'lng', 'mday',
       'sarea', 'sareaen', 'sbi', 'sna', 'snaen', 'sno', 'srcUpdateTime',
       'tot', 'updateTime', 'thres', 'station_status'],
      dtype='object')

In [149]:
ubike_rt = ubike_rt[['sna','sbi','bemp','station_status','infoDate','infoTime']]

In [150]:
# aval_bike: int
# aval_space: int
# station_status: str
# date: time
# hour: int
# weekend: str
# distance: int

In [151]:
ubike_rt = ubike_rt.rename(
    {
        'sna':'Bike_station_name',
        'sbi':'aval_bike',
        'bemp':'aval_space',
        'infoDate':'date'
    },axis=1)

In [152]:
ubike_rt['hour'] = pd.to_datetime(ubike_rt['infoTime']).dt.hour
ubike_rt['min'] = pd.to_datetime(ubike_rt['infoTime']).dt.minute

In [153]:
ubike_rt['minutes_aligned'] = ubike_rt['min']//10*10

In [154]:
ubike_rt['weekend'] = pd.to_datetime(ubike_rt['date']).dt.dayofweek.isin([5,6])
ubike_rt['weekend'] = np.where(ubike_rt['weekend'],'假日','平日')

In [155]:
ubike_rt = ubike_rt.drop(['infoTime','min'],axis=1)

In [158]:
ubike_rt[ubike_rt.duplicated()]

,Bike_station_name,aval_bike,aval_space,station_status,date,hour,minutes_aligned,weekend
act,,,,,,,,
1,和平公園東側,3,8,正常租借,2024-04-16,22,20,平日
1,辛亥復興路口西北側,2,14,正常租借,2024-04-16,22,20,平日
1,新生南路三段66號前,0,16,無車可借,2024-04-16,22,20,平日
1,新生南路三段82號前,0,20,無車可借,2024-04-16,22,20,平日
1,辛亥路一段30號前,0,8,無車可借,2024-04-16,22,10,平日
...,...,...,...,...,...,...,...,...
1,臺大原分所北側,0,12,無車可借,2024-05-01,19,20,平日
1,臺大森林館北側,6,4,正常租借,2024-05-01,19,40,平日
1,臺大教研館北側,10,1,有車少位,2024-05-01,19,40,平日


In [160]:
print(ubike_rt.shape)
ubike_rt = ubike_rt.drop_duplicates(subset=['Bike_station_name','date','hour','minutes_aligned'])
print(ubike_rt.shape)


(2620014, 8)
(1309354, 8)


In [166]:
ubike_rt['date'].max()

'2024-05-01'

In [165]:
ubike_rt['date'].min()

'2024-01-14'

In [176]:
ubike_rt['date'].sort_values().unique()

array(['2024-01-14', '2024-03-02', '2024-03-15', '2024-03-18',
       '2024-03-26', '2024-03-30', '2024-03-31', '2024-04-09',
       '2024-04-16', '2024-04-17', '2024-04-18', '2024-04-19',
       '2024-04-20', '2024-04-21', '2024-04-22', '2024-04-23',
       '2024-04-24', '2024-04-25', '2024-04-26', '2024-04-27',
       '2024-04-28', '2024-04-29', '2024-04-30', '2024-05-01'],
      dtype=object)

In [177]:
ubike_rt = ubike_rt[~ubike_rt['date'].isin(['2024-01-14', '2024-03-02', '2024-03-15', '2024-03-18',
       '2024-03-26', '2024-03-30', '2024-03-31', '2024-04-09'])]

In [162]:
ubike_rt = ubike_rt.merge(bike_station,how='inner',on='Bike_station_name')

In [179]:
ubike_rt.to_csv('pipeline_low_avail.csv',index=False)